In [1]:
import pandas as pd
import pysubgroup as ps
from os import path

In [4]:
data = pd.read_csv("graph_features.csv")
data.index = data["Unnamed: 0"]
data = data.drop(columns="Unnamed: 0")
data.head(10)

,average_degree,density,longest_path,degree_centralization,betweenness_centralization,003,012,021D,021U,021C,030T,dominant_topic
Unnamed: 0,,,,,,,,,,,,
805443722,1.971119,0.003571,0.039711,0.233676,0.000489,12387.747292,253.530686,0.021661,8.061372,0.584838,0.054152,4
805452109,1.984615,0.003831,0.042308,0.268534,0.000792,10892.280769,233.453846,0.396154,10.180769,0.657692,0.030769,1
805453752,2.032787,0.003343,0.029508,0.311143,0.000982,15060.629508,274.786885,0.111475,15.819672,0.639344,0.013115,4
805455311,2.139241,0.003396,0.056962,0.188050,0.001200,16156.401899,321.373418,0.082278,6.177215,0.927215,0.037975,4
805455361,2.251572,0.003551,0.031447,0.313481,0.000521,16357.632075,319.694969,0.707547,16.279874,0.977987,0.040881,4
805459722,2.058442,0.003353,0.042208,0.478487,0.000623,15378.159091,242.759740,0.016234,35.480519,0.564935,0.019481,2
805466053,1.927419,0.003902,0.064516,0.155393,0.001369,9894.221774,228.540323,0.181452,3.350806,0.649194,0.056452,0
805467158,1.767606,0.003123,0.066901,0.050648,0.002721,13053.524648,245.739437,0.056338,1.193662,0.464789,0.021127,4
805467799,1.681356,0.002859,0.020339,0.042186,0.000200,14112.169492,243.355932,0.006780,1.145763,0.308475,0.013559,1


## Search for subgroups in the whole data set

In [5]:
ps_target = ps.NumericTarget('dominant_topic')
ps_search_space = ps.create_selectors(data, ignore=['dominant_topic'])
ps_task = ps.SubgroupDiscoveryTask(data, ps_target, ps_search_space, qf=ps.StandardQFNumeric(1))
result = ps.DFSNumeric().execute(ps_task)


In [11]:
result.to_dataframe()

,quality,description
0,520.504171,030T: [0.01:0.03[
1,460.417362,021U: [4.78:11.56[
2,450.417362,degree_centralization: [0.17:0.27[
3,412.417362,030T<0.01
4,405.873957,021D>=0.21
5,405.085882,021U: [4.78:11.56[ AND degree_centralization: ...
6,367.873957,003>=15432.11
7,345.873957,degree_centralization>=0.27
8,335.873957,021U>=11.56
9,316.198641,021U>=11.56 AND degree_centralization>=0.27


## Search for subgroups within the topics found by LDA

In [9]:
result_dict = {}
ps_target = ps.NumericTarget('dominant_topic')
for topic_no in data.dominant_topic.unique():
    topic_data = data[data.dominant_topic == topic_no]
    ps_search_space = ps.create_nominal_selectors(topic_data, ignore=['dominant_topic'])
    ps_task = ps.SubgroupDiscoveryTask(topic_data, ps_target, ps_search_space, qf=ps.StandardQFNumeric(1))
    result_dict[topic_no] = ps.DFSNumeric().execute(ps_task)

In [10]:
for topic_no, subgroup_result in result_dict.items():
    print(f"Result for topic no {topic_no}: {subgroup_result.to_dataframe()}")

Result for topic no 4: Empty DataFrame
Columns: [quality, description]
Index: []
Result for topic no 1: Empty DataFrame
Columns: [quality, description]
Index: []
Result for topic no 2: Empty DataFrame
Columns: [quality, description]
Index: []
Result for topic no 0: Empty DataFrame
Columns: [quality, description]
Index: []
Result for topic no 3: Empty DataFrame
Columns: [quality, description]
Index: []


## Reproducing the bug

With passing a integer to StandardQFNumeric:

In [7]:
ps_target = ps.NumericTarget('dominant_topic')
ps_search_space = ps.create_selectors(data, ignore=['dominant_topic'])
ps_task = ps.SubgroupDiscoveryTask(data, ps_target, ps_search_space, qf=ps.StandardQFNumeric(1))
result = ps.BeamSearch().execute(ps_task)

TypeError: unsupported operand type(s) for -: 'method' and 'float'